# Информационный поиск

Скачиваем классический набор данных -- набор текстов об аэронавтике CRANFIELD

In [5]:
! wget -q http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
! tar -xvf cran.tar.gz
! rm cran.tar.gz*

cran.all.1400
cran.qry
cranqrel
cranqrel.readme


Берём только сами запросы (это будут наши документы)

In [6]:
! grep -v "^\." cran.qry > just.qry
! head -3 just.qry

what similarity laws must be obeyed when constructing aeroelastic models
of heated high speed aircraft .
what are the structural and aeroelastic problems associated with flight


Объединяем многострочные в один

In [7]:
raw_query_data = [line.strip() for line in open("just.qry", "r").readlines()]
query_data = [""]

for query_part in raw_query_data:
  query_data[-1] += query_part + " "
  if query_part.endswith("."):
    query_data.append("")

query_data[:2] #Выведем пару документов для примера

['what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . ',
 'what are the structural and aeroelastic problems associated with flight of high speed aircraft . ']

### Составим запросы к нашим документам

In [25]:
# QUERIES = ['theory of bending', 'aeroelastic effects']
QUERIES = ['unsteady aerodynamics']


## Boolean retrieval
Представим каждый документ как "битовую маску": вектор размером со словарь, в котором на каждой позиции единица, если в документе есть соответсвующий терм, и ноль, если терма нет

In [13]:
# в разных версиях ответы могут отличаться, поэтому важно иметь одну и ту же
! pip install -q scikit-learn==0.22.2.post1

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for scikit-learn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (scikit-learn)


In [26]:
from  sklearn.feature_extraction.text import CountVectorizer

encoder = CountVectorizer(binary=True)
encoded_data = encoder.fit_transform(query_data)
encoded_queries = encoder.transform(QUERIES)
list(encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

Посмотрим на представление первого предложения

In [27]:
id2term = {idx: term for term, idx in encoder.vocabulary_.items()}
non_zero_values_ids = encoded_data[0].nonzero()[1]

terms = [id2term[idx] for idx in non_zero_values_ids]
terms

['what',
 'similarity',
 'laws',
 'must',
 'be',
 'obeyed',
 'when',
 'constructing',
 'aeroelastic',
 'models',
 'of',
 'heated',
 'high',
 'speed',
 'aircraft']

Всё так.

## Задание 0

Теперь для каждого из данных запросов `QUERIES` найдём ближайший для него документ из `query_data` по сходству Жаккара. Есть более эффективные способы это сделать, но вам требуется реализовать расстояние Жаккара и далее применить его к нашим данным.

In [28]:
import numpy as np

def jaccard_sim(vector_a: np.array, vector_b: np.array) -> float:
  """
    Сходство или коэффициент Жаккара: отношение мощности пересечения
    к мощности объединения
  """
  # ваш код здесь
  intersection = np.logical_and(vector_a, vector_b).sum()
  union = np.logical_or(vector_a, vector_b).sum()

  return 0.0 if union == 0 else intersection / union
#Проверка, что функция работает правильно
assert jaccard_sim(np.array([1, 0, 1, 0, 1]), np.array([0, 1, 1, 1, 1])) == 0.4

Здесь документы представлены так же, как строки в матрице термов-документов. Каждая ячейка вектора отвечает за наличие/отсутствие конкретного элемента (например, слова-терма, когда у нас в словаре всего 5 слов). В первом случае их три, во втором — четыре. Объединение — все пять возможных элементов. Пересечение — два. Отсюда и 0.4.

## Задание 1
Теперь с помощью кода ниже вычислите для каждого запроса самые близкие документы.

In [30]:
for q_id, query in enumerate(encoded_queries):
  # приводим к нужному типу
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in encoded_data]
  # вычисляем коэфф. Жаккара
  id2doc2similarity = [(doc_id, doc, jaccard_sim(query, doc)) for doc_id, doc in enumerate(docs)]
  # сортируем по нему
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=True)

  print("Q: %s\nFOUND:" % QUERIES[q_id])
  # выводим по 3 наиболее близких документа для каждого запроса
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))



Q: unsteady aerodynamics
FOUND:
    22	0.20	what progress has been made in research on unsteady aerodynamics . 
    95	0.07	has anyone investigated the unsteady lift distributions on finite wings in subsonic flow . 
    0	0.00	what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . 


Видим, что кое-где просачиваются  тексты, которых с запросами объединяют малозначительные термы, но при этом коэффициент Жаккара -- наша функция ранжирования! -- высок.

# VSM

Попробуем теперь сделать то же, но с tf-idf и косинусным расстоянием. Мы сделаем всё опять "руками", но "в реальной жизни" лучше использоватьесть эффективные реализации cosine distance, например, из библиотеки scipy.

In [31]:
from  sklearn.feature_extraction.text import TfidfVectorizer

# Совет: обязательно разберитесь с тем, какие возможности
# предоставляет tf-idf vectorizer, какие параметры за что отвечают

tfidf_encoder = TfidfVectorizer()
tfidf_encoded_data = tfidf_encoder.fit_transform(query_data)
tfidf_encoded_queries = tfidf_encoder.transform(QUERIES)

list(tfidf_encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

## Задание 2
Реализовать косинусное расстояние

In [32]:
import numpy as np

def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
  """
    Косинусное расстояние: единица минус отношение скалярного произведения
    на произведение L2-норм (подсказка: в numpy такие нормы есть)
  """
  # ваш код здесь
  cos_dist = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
  cos_dist = 0.0 if cos_dist == 0.0 else np.dot(vector_a, vector_b) / cos_dist
  cos_dist = 1 - cos_dist

  return cos_dist
#Проверка, что функция работает правильно
assert cosine_distance(np.array([1, 0, 1, 1, 1]), np.array([0, 0, 1, 0, 0])) == 0.5


Теперь вычислим ближайшие по косинусному расстоянию между векторными представлениями документов и запросов

In [33]:
for q_id, query in enumerate(tfidf_encoded_queries):

  # приводим к нужному типу
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in tfidf_encoded_data]
  # Косинусное расстояние
  id2doc2similarity = [(doc_id, doc, cosine_distance(query, doc)) \
                       for doc_id, doc in enumerate(docs)]
  # сортируем по нему
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=False)

  print("Q: %s\nFOUND:" % QUERIES[q_id])

  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Q: unsteady aerodynamics
FOUND:
    22	0.41	what progress has been made in research on unsteady aerodynamics . 
    95	0.74	has anyone investigated the unsteady lift distributions on finite wings in subsonic flow . 
    0	1.00	what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . 
